In [1]:
from xml.dom import minidom

## Read XML file

In [2]:
annot = minidom.parse("./data/annot.annotations") # open XML file

In [9]:
regions = annot.getElementsByTagName("Region") # list of regions in the annotation file
print(len(regions))

26


In [10]:
V = [] # list of vertices in each region
V_coord = [] # list of coord of each vertex in each region
for r in regions :
    vertices = r.getElementsByTagName("V")
    V.append(vertices)
    
    v_coord = []
    for v in vertices :
        coord = {"X": v.getAttribute('X'), "Y": v.getAttribute('Y')}
        v_coord.append(coord)
    V_coord.append(v_coord)

In [11]:
V_coord

[[{'X': '2667', 'Y': '16944'},
  {'X': '2667', 'Y': '16967'},
  {'X': '2667', 'Y': '17058'},
  {'X': '2576', 'Y': '17422'},
  {'X': '2440', 'Y': '18104'},
  {'X': '2394', 'Y': '18445'},
  {'X': '2394', 'Y': '18627'},
  {'X': '2394', 'Y': '18900'},
  {'X': '2394', 'Y': '19286'},
  {'X': '2394', 'Y': '19696'},
  {'X': '2394', 'Y': '20082'},
  {'X': '2303', 'Y': '20423'},
  {'X': '2281', 'Y': '20651'},
  {'X': '2190', 'Y': '20946'},
  {'X': '2099', 'Y': '21242'},
  {'X': '2112', 'Y': '21410'},
  {'X': '2112', 'Y': '21545'},
  {'X': '2090', 'Y': '21613'},
  {'X': '2090', 'Y': '21725'},
  {'X': '2090', 'Y': '21793'},
  {'X': '2090', 'Y': '21838'},
  {'X': '2090', 'Y': '21928'},
  {'X': '2067', 'Y': '22041'},
  {'X': '2045', 'Y': '22109'},
  {'X': '2022', 'Y': '22176'},
  {'X': '2000', 'Y': '22221'},
  {'X': '1932', 'Y': '22312'},
  {'X': '1909', 'Y': '22334'},
  {'X': '1864', 'Y': '22424'},
  {'X': '1842', 'Y': '22447'},
  {'X': '1842', 'Y': '22515'},
  {'X': '1774', 'Y': '22673'},
  {'X': 

#### 0/1 array representation of regions of interest

In [1]:
# Images
import os
import PIL 
import numpy as np
import openslide

# Annotations
from xml.dom import minidom
import skimage.draw

In [17]:
img = np.ones((1008, 840)) # creates an array of ones of the same shape of our image
mask_img = np.zeros((1008, 840, 3), dtype=np.uint8)
ds_rate = 64 # downsample rate

polygons = [] # list of polygons, each object is an array containing the coord of the px in a region of V_coord

for region in V_coord:
    x = [] # list of x coord in the region
    y = [] # list of y coord in the region

    for v in region:
        # as we dont take the full resolution slide of the image, we need to divide the coord of each vertice by the downsampling rate
        temp_x = int(v['X'])/ds_rate 
        temp_y = int(v['Y'])/ds_rate

        x.append(int(temp_x))
        y.append(int(temp_y))

    # sikimage computes the coord of each px in the region from the vertices defining the perimeter of the region
    polygons.append(skimage.draw.polygon(x,y)) 

In [2]:
def png_mask(img_shape, ds_rate, V_coord):
    mask = np.zeros((img_shape[0], img_shape[1], 3), dtype=np.uint8) # creates an array of ones of the same shape of our image
    #ds_rate = 8 # downsample rate
    print(mask.shape)
    # list of polygons, each object is an array containing the coords of the px in a region of V_coord
    polygons = [] # 1 polygon / region

    for region in V_coord:
        x = [] # list of x coord in the region
        y = [] # list of y coord in the region

        # fill the x and y coord arrays
        for v in region:
            # as we dont take the full resolution slide of the image, 
            # we need to divide the coord of each vertice by the downsampling rate
            temp_x = int(v['X'])/ds_rate 
            temp_y = int(v['Y'])/ds_rate

            x.append(int(temp_x))
            y.append(int(temp_y))

        # sikimage computes the coord of each px in the region from the vertices defining 
        # the perimeter of the region
        polygons.append(skimage.draw.polygon(x,y))
        print(len(polygons[0][0]))
        print(len(polygons[0][1]))
        print(polygons)
        
    # modify mask so that each px in a region of V_coord is set to 0
    for p in range(len(polygons)):
        poly = np.transpose(polygons[p])

        for i in range(len(poly)):
            print(poly[i][0], poly[i][1])
            mask[(poly[i][0], poly[i][1])] = [255, 255, 255]
        
    return mask

In [4]:
import pipeline_IHC as pipe
directory = "./test_data/test_png/"
image = pipe.load_img(directory)
name = "HN4"
xml = minidom.parse("./test_data/test_png/HN4.annotations")
coords = pipe.xml_to_vertices(xml, '0')
img_shape = image[name].shape
print(img_shape)
#mask_img = png_mask(img_shape, 64, coords)
pipe.load_annot(directory,image)

(357, 574, 4)
85 269
85 270
85 271
85 272
85 273
85 274
86 267
86 268
86 269
86 270
86 271
86 272
86 273
86 274
86 275
86 276
86 277
87 266
87 267
87 268
87 269
87 270
87 271
87 272
87 273
87 274
87 275
87 276
87 277
87 278
87 279
88 265
88 266
88 267
88 268
88 269
88 270
88 271
88 272
88 273
88 274
88 275
88 276
88 277
88 278
88 279
88 280
89 264
89 265
89 266
89 267
89 268
89 269
89 270
89 271
89 272
89 273
89 274
89 275
89 276
89 277
89 278
89 279
89 280
89 281
90 264
90 265
90 266
90 267
90 268
90 269
90 270
90 271
90 272
90 273
90 274
90 275
90 276
90 277
90 278
90 279
90 280
90 281
90 282
91 263
91 264
91 265
91 266
91 267
91 268
91 269
91 270
91 271
91 272
91 273
91 274
91 275
91 276
91 277
91 278
91 279
91 280
91 281
91 282
91 283
92 262
92 263
92 264
92 265
92 266
92 267
92 268
92 269
92 270
92 271
92 272
92 273
92 274
92 275
92 276
92 277
92 278
92 279
92 280
92 281
92 282
92 283
92 284
93 261
93 262
93 263
93 264
93 265
93 266
93 267
93 268
93 269
93 270
93 271
93 272
93 273

120 242
120 243
120 244
120 245
120 246
120 247
120 248
120 249
120 250
120 251
120 252
120 253
120 254
120 255
120 256
120 257
120 258
120 259
120 260
120 261
120 262
120 263
120 264
120 265
120 266
120 267
120 268
120 269
120 270
120 271
120 272
120 273
120 274
120 275
120 276
120 277
120 278
120 279
120 280
120 281
120 282
120 283
120 284
120 285
120 286
120 287
120 288
120 289
120 290
120 291
120 292
120 293
120 294
120 295
120 296
120 297
120 298
120 299
121 238
121 239
121 240
121 241
121 242
121 243
121 244
121 245
121 246
121 247
121 248
121 249
121 250
121 251
121 252
121 253
121 254
121 255
121 256
121 257
121 258
121 259
121 260
121 261
121 262
121 263
121 264
121 265
121 266
121 267
121 268
121 269
121 270
121 271
121 272
121 273
121 274
121 275
121 276
121 277
121 278
121 279
121 280
121 281
121 282
121 283
121 284
121 285
121 286
121 287
121 288
121 289
121 290
121 291
121 292
121 293
121 294
121 295
121 296
121 297
121 298
121 299
121 300
122 239
122 240
122 241
122 242


153 314
153 315
153 316
154 249
154 250
154 251
154 252
154 253
154 254
154 255
154 256
154 257
154 258
154 259
154 260
154 261
154 262
154 263
154 264
154 265
154 266
154 267
154 268
154 269
154 270
154 271
154 272
154 273
154 274
154 275
154 276
154 277
154 278
154 279
154 280
154 281
154 282
154 283
154 284
154 285
154 286
154 287
154 288
154 289
154 290
154 291
154 292
154 293
154 294
154 295
154 296
154 297
154 298
154 299
154 300
154 301
154 302
154 303
154 304
154 305
154 306
154 307
154 308
154 309
154 310
154 311
154 312
154 313
154 314
154 315
154 316
154 317
155 248
155 249
155 250
155 251
155 252
155 253
155 254
155 255
155 256
155 257
155 258
155 259
155 260
155 261
155 262
155 263
155 264
155 265
155 266
155 267
155 268
155 269
155 270
155 271
155 272
155 273
155 274
155 275
155 276
155 277
155 278
155 279
155 280
155 281
155 282
155 283
155 284
155 285
155 286
155 287
155 288
155 289
155 290
155 291
155 292
155 293
155 294
155 295
155 296
155 297
155 298
155 299
155 300


177 281
177 282
177 283
177 284
177 285
177 286
177 287
177 288
177 289
177 290
177 291
177 292
177 293
177 294
177 295
177 296
177 297
177 298
177 299
177 300
177 301
177 302
177 303
177 304
177 305
177 306
177 307
177 308
177 309
177 310
177 311
177 312
177 313
177 314
177 315
177 316
177 317
177 318
177 319
177 320
177 321
177 322
177 323
177 324
177 325
177 326
177 327
177 328
178 235
178 236
178 237
178 238
178 239
178 240
178 241
178 242
178 243
178 244
178 245
178 246
178 247
178 248
178 249
178 250
178 251
178 252
178 253
178 254
178 255
178 256
178 257
178 258
178 259
178 260
178 261
178 262
178 263
178 264
178 265
178 266
178 267
178 268
178 269
178 270
178 271
178 272
178 273
178 274
178 275
178 276
178 277
178 278
178 279
178 280
178 281
178 282
178 283
178 284
178 285
178 286
178 287
178 288
178 289
178 290
178 291
178 292
178 293
178 294
178 295
178 296
178 297
178 298
178 299
178 300
178 301
178 302
178 303
178 304
178 305
178 306
178 307
178 308
178 309
178 310
178 311


198 309
198 310
198 311
198 312
198 313
198 314
198 315
198 316
198 317
198 318
198 319
198 320
198 321
198 322
198 323
198 324
198 325
198 326
198 327
198 328
198 329
198 330
198 331
198 332
198 333
198 334
198 335
198 336
198 337
198 338
198 339
198 340
198 341
198 342
198 343
198 344
198 345
198 346
198 347
198 348
198 349
198 350
198 351
198 352
198 353
199 232
199 233
199 234
199 235
199 236
199 237
199 238
199 239
199 240
199 241
199 242
199 243
199 244
199 245
199 246
199 247
199 248
199 249
199 250
199 251
199 252
199 253
199 254
199 255
199 256
199 257
199 258
199 259
199 260
199 261
199 262
199 263
199 264
199 265
199 266
199 267
199 268
199 269
199 270
199 271
199 272
199 273
199 274
199 275
199 276
199 277
199 278
199 279
199 280
199 281
199 282
199 283
199 284
199 285
199 286
199 287
199 288
199 289
199 290
199 291
199 292
199 293
199 294
199 295
199 296
199 297
199 298
199 299
199 300
199 301
199 302
199 303
199 304
199 305
199 306
199 307
199 308
199 309
199 310
199 311


...

330 300
330 301
330 302
330 303
330 304
330 305
330 306
330 307
330 308
330 309
330 310
330 311
330 312
330 313
330 314
330 315
330 316
330 317
330 318
330 319
330 320
330 321
330 322
330 323
330 324
330 325
330 326
330 327
330 328
330 329
330 330
330 331
330 332
330 333
330 334
330 335
330 336
330 337
330 338
330 339
330 340
330 341
330 342
330 343
330 344
330 345
330 346
330 347
330 348
330 349
330 350
330 351
330 352
330 353
330 354
330 355
330 356
330 357
330 358
330 359
330 360
330 361
330 362
330 363
330 364
330 365
330 366
330 367
330 368
330 369
330 370
330 371
330 372
330 373
330 374
330 375
330 376
330 377
330 378
330 379
330 380
330 381
330 382
330 383
330 384
330 385
330 386
330 387
330 388
330 389
330 390
330 391
330 392
330 393
330 394
330 395
330 396
330 397
330 398
330 399
330 400
330 401
330 402
330 403
330 404
330 405
330 406
330 407
330 408
330 409
330 410
330 411
330 412
330 413
330 414
330 415
330 416
330 417
330 418
330 419
330 420
330 421
330 422
330 423
330 424


338 262
338 263
338 264
338 265
338 266
338 267
338 268
338 269
338 270
338 271
338 272
338 273
338 274
338 275
338 276
338 277
338 278
338 279
338 280
338 281
338 282
338 283
338 284
338 285
338 286
338 287
338 288
338 289
338 290
338 291
338 292
338 293
338 294
338 295
338 296
338 297
338 298
338 299
338 300
338 301
338 302
338 303
338 304
338 305
338 306
338 307
338 308
338 309
338 310
338 311
338 312
338 313
338 314
338 315
338 316
338 317
338 318
338 319
338 320
338 321
338 322
338 323
338 324
338 325
338 326
338 327
338 328
338 329
338 330
338 331
338 332
338 333
338 334
338 335
338 336
338 337
338 338
338 339
338 340
338 341
338 342
338 343
338 344
338 345
338 346
338 347
338 348
338 349
338 350
338 351
338 352
338 353
338 354
338 355
338 356
338 357
338 358
338 359
338 360
338 361
338 362
338 363
338 364
338 365
338 366
338 367
338 368
338 369
338 370
338 371
338 372
338 373
338 374
338 375
338 376
338 377
338 378
338 379
338 380
338 381
338 382
338 383
338 384
338 385
338 386


345 391
345 392
345 393
345 394
345 395
345 396
345 397
345 398
345 399
345 400
345 401
345 402
345 403
345 404
345 405
345 406
345 407
345 408
345 409
345 410
345 411
345 412
345 413
345 414
345 415
345 416
345 417
345 418
345 419
345 420
345 421
345 422
345 423
345 424
345 425
345 426
345 427
345 428
345 429
345 430
345 431
345 432
345 433
345 434
345 435
345 436
345 437
345 438
345 439
345 440
345 441
345 442
345 443
345 444
345 445
345 446
345 447
345 448
345 449
345 450
345 451
345 452
345 453
345 454
345 455
345 456
345 457
346 263
346 264
346 265
346 266
346 267
346 268
346 269
346 270
346 271
346 272
346 273
346 274
346 275
346 276
346 277
346 278
346 279
346 280
346 281
346 282
346 283
346 284
346 285
346 286
346 287
346 288
346 289
346 290
346 291
346 292
346 293
346 294
346 295
346 296
346 297
346 298
346 299
346 300
346 301
346 302
346 303
346 304
346 305
346 306
346 307
346 308
346 309
346 310
346 311
346 312
346 313
346 314
346 315
346 316
346 317
346 318
346 319
346 320


352 377
352 378
352 379
352 380
352 381
352 382
352 383
352 384
352 385
352 386
352 387
352 388
352 389
352 390
352 391
352 392
352 393
352 394
352 395
352 396
352 397
352 398
352 399
352 400
352 401
352 402
352 403
352 404
352 405
352 406
352 407
352 408
352 409
352 410
352 411
352 412
352 413
352 414
352 415
352 416
352 417
352 418
352 419
352 420
352 421
352 422
352 423
352 424
352 425
352 426
352 427
352 428
352 429
352 430
352 431
352 432
352 433
352 434
352 435
352 436
352 437
352 438
352 439
352 440
352 441
352 442
352 443
352 444
352 445
352 446
352 447
352 448
352 449
352 450
352 451
352 452
352 453
352 454
352 455
352 456
352 457
352 458
352 475
352 476
352 477
352 478
352 479
352 480
352 481
352 482
353 268
353 269
353 270
353 271
353 272
353 273
353 274
353 275
353 276
353 277
353 278
353 279
353 280
353 281
353 282
353 283
353 284
353 285
353 286
353 287
353 288
353 289
353 290
353 291
353 292
353 293
353 294
353 295
353 296
353 297
353 298
353 299
353 300
353 301
353 302


IndexError: index 357 is out of bounds for axis 0 with size 357

In [21]:
PILimg = PIL.Image.fromarray(mask_img, 'RGB')
PILimg.save('mask.png')
PILimg.show()

In [18]:
# modify img so that each px in a region of V_coord is set to 0
for p in range(len(polygons)):
    poly = np.transpose(polygons[p])

    for i in range(len(poly)):
        mask_img[(poly[i][0], poly[i][1])] = [255, 255, 255]
        img[(poly[i][0], poly[i][1])] = 0

In [19]:
mask_img.shape

(1008, 840, 3)

In [20]:
PILimg = PIL.Image.fromarray(mask_img, 'RGB')
PILimg.save('my.png')
PILimg.show()

## Write XML file

In [6]:
new_annot = minidom.Document() # create a new file

annotations = new_annot.createElement("Annotations") # create element Annotations

annotation = new_annot.createElement("Annotation") # create element Annotation
# set all the attributes 
annotation.setAttribute('LineColor',"65535") 
annotation.setAttribute('Name', "Layer 1")
annotation.setAttribute('Visible', "True")

annotations.appendChild(annotation) # add Annotation to Annotations

<DOM Element: Annotation at 0x7f95d9d47550>

In [7]:
for R in V_coord:
    new_region = new_annot.createElement("Region") # create a new region
    for V in R:
        new_vertice = new_annot.createElement("V") # create a new vertice
        # set vertice's coord
        new_vertice.setAttribute('X', V['X'])
        new_vertice.setAttribute('Y', V['Y'])
        
        new_region.appendChild(new_vertice) # add vertice to region
        
    annotation.appendChild(new_region) # add region to annotation

In [8]:
new_annot.appendChild(annotations) # add Annotations to our document

<DOM Element: Annotations at 0x7f95d9d473a0>

In [9]:
myfile = open("./data/annot_new.annotations", "w") # open a new file
myfile.write(new_annot.toprettyxml()) # write the content of our document in XML in the file
myfile.close()